In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [47]:
file_name = "/Users/dafirebanks/Downloads/SongCSV.csv"
df = pd.read_csv(file_name)

In [5]:
df.columns

Index(['SongNumber', 'SongID', 'AlbumID', 'AlbumName', 'ArtistFamiliarity',
       'ArtistHotttnesss', 'ArtistID', 'ArtistLatitude', 'ArtistLocation',
       'ArtistLongitude', 'ArtistName', 'BarsConfidence', 'BarsStart',
       'BeatsConfidence', 'BeatsStart', 'Danceability', 'Duration',
       'EndOfFadeIn', 'Energy', 'KeySignature', 'KeySignatureConfidence',
       'Loudness', 'Mode', 'ModeConfidence', 'SectionsConfidence',
       'SectionsStart', 'SegmentsConfidence', 'SegmentsLoudnessMax',
       'SegmentsLoudnessMaxTime', 'SegmentsLoudnessMaxStart',
       'SegmentsPitches', 'SegmentsStart', 'SegmentsTimbre', 'SongHotttnesss',
       'TatumsConfidence', 'TatumsStart', 'Tempo', 'TimeSignature',
       'TimeSignatureConfidence', 'Title', 'Year'],
      dtype='object')

In [79]:
df.head()

,SongNumber,SongID,AlbumID,AlbumName,ArtistFamiliarity,ArtistHotttnesss,ArtistID,ArtistLatitude,ArtistLocation,ArtistLongitude,...,SegmentsStart,SegmentsTimbre,SongHotttnesss,TatumsConfidence,TatumsStart,Tempo,TimeSignature,TimeSignatureConfidence,Title,Year
0,1,b'SOGSMXL12A81C23D88',275907,b'Je Sais Que La Terre Est Plate (Deluxe)',0.557460,0.386152,b'AREJXK41187B9A4ACC',46.71067,b'France',1.71819,...,[ 0. 0.19188 0.60499 0.84939 1.1...,[[ 0. 171.13 9.469 ... 0.973 -10...,0.547953,[0. 0. 0. 0. 0. 0. 0. 0. ...,[4.2570000e-02 2.8357000e-01 5.2458000e-01 7.6...,124.059,4,0.372,b'Je Sais Que La Terre Est Plate',2008
1,2,b'SOMBCOW12AAF3B229F',382807,b'Comme Vous',0.626958,0.434860,b'AR2XRFQ1187FB417FE',NaN,b'',NaN,...,[ 0. 0.51379 0.89138 1.30912 1.6...,[[ 8.00000e-03 1.70986e+02 9.12600e+00 ... ...,0.475638,[0.365 0.321 0.29 0.264 0.238 0.217 0.208 0.1...,[ 0.59161 0.84042 1.08799 1.33555 1.5...,80.084,4,0.533,b'On Efface',2004
2,3,b'SOEYIHF12AB017B5F4',490659,b'Watkins Ale - Music of the English Renaissa...,0.425724,0.000000,b'ARODOO01187FB44F4A',NaN,b'',NaN,...,[ 0. 0.97365 1.34431 1.81351 2.19764 ...,[[ 1.337 145.247 -43.274 ... 16.283 -13...,NaN,[0.307 0.305 0.291 0.284 0.281 0.28 0.276 0.2...,[ 1.11537 1.39701 1.67864 1.95344 2.23097 ...,54.874,4,0.000,b'Howells Delight',0
3,4,b'SODJYEC12A8C13D757',116616,"bDon't Worry Lady""""",0.611495,0.334520,b'ARJGW911187FB586CA',NaN,b'',NaN,...,[0.0000000e+00 7.3060000e-02 3.6862000e-01 6.4...,[[ 14.832 -56.33 69.981 ... 92.933 29...,NaN,[0.731 0.628 0.526 0.359 0.287 0.246 0.21 0.1...,[1.1929000e-01 3.9309000e-01 6.5603000e-01 9.1...,77.150,3,0.369,b'Martha Served',2007
4,5,b'SOGSOUE12A58A76443',767122,b'Easy Listening: Cartoon Songs',0.367255,0.311616,b'AR9HQ6Y1187FB3C2CB',NaN,b'',NaN,...,[ 0. 0.3166 0.82195 1.2751 1.8...,[[ 10.095 38.901 84.505 ... 53.882 -30.045 ...,NaN,[0. 0. 0. 0. 0. 0. 0. 0. ...,[ 0.24248 0.49266 0.74283 0.99301 1.2...,120.382,4,1.000,b'Zip-A-Dee-Doo-Dah (Song of the South)',0


In [73]:
#### COLUMNS THAT ARE LISTS 
# Bars Confidence (99,)
# Bars Start (99,)

# Beats Confidence (397,)
# Beats Start (397,)

# Sections Confidence (10,)
# Sections Start (10,)

# SegmentsConfidence (935,)
# SegmentsLoudness max time (935,)
# SegmentsLoudness start (935, )
# Segments Start (935,)

# SegmentsPitches (935, 12)
# Segments Timbre (935, 12)

# Tatums confidence (794,)
# Tatums Start (794,)

In [48]:
list_columns = ["BarsConfidence", "BarsStart", "BeatsConfidence", "BeatsStart", "SectionsConfidence", "SectionsStart", 
                "SegmentsConfidence", "SegmentsLoudnessMax", "SegmentsLoudnessMaxTime", "SegmentsLoudnessMaxStart", 
                "SegmentsStart", "TatumsConfidence", "TatumsStart"]
    
def transform_str_to_list(df, columns):
    for col in columns:
        all_cells = []
        
        # Modify each cell 
        for i in range(0, len(df[col])):
            
            # Get list
            horrid_string = df[col][i]
            
            # Remove non-float values
            pretty_string = horrid_string.replace("[", "").replace("]", "").replace("...", "")
            
            # Convert to a list of floats
            pretty_list = pretty_string.split()
            try:
                beautiful_list = np.array([float(el) for el in pretty_list])
            except:
                print(col, i)
                print(pretty_list)
            
            # Add to list of cells
            all_cells.append(beautiful_list)
        
        # Make it a new column
        df[col] = pd.Series(all_cells)

In [49]:
transform_str_to_list(df, list_columns)

In [ ]:
df = df.drop(columns=["Energy", "SegmentsPitches", "SegmentsTimbre"])

In [62]:
df["BarsConfidence"][954]

array([0., 0.])

In [71]:
# Transform all arrays to the minimum
def reduce_arrays(col_name):
    col = df[col_name]
    
    min_len = len(col[0])
    
    # Get minimum length 
    for i in range(len(col)):
        cell = col[i]
        
        # Check that the list is not empty and that all the elements are not 0
        if len(cell) < min_len and len(cell) != 0 and not all(v == 0.0 for v in cell):    
            min_len = len(cell)
            print(i, len(cell), cell)

    print("Minimum length is", min_len)
        
    # Transform all arrays in the column to the minimum length
    for i in range(len(col)):
        cell = col[i]
        
        # If we stumble upon a zero array or an empty array...
        if len(cell) == 0 or len(cell) < min_len:
            col[i] = np.zeros(min_len)
        else: # We get the first min_len elements
            col[i] = col[i][:min_len]
    
    return col

In [ ]:
test = 

In [42]:
df.iloc[33]

SongNumber                                                                 34
SongID                                                  b'SOFIYUN12AB0184842'
AlbumID                                                                569832
AlbumName                                                 b'Afraid Of Change'
ArtistFamiliarity                                                     0.20073
ArtistHotttnesss                                                            0
ArtistID                                                b'ARSQFIK11F50C4D038'
ArtistLatitude                                                            NaN
ArtistLocation                                                            b''
ArtistLongitude                                                           NaN
ArtistName                                                      b'Casa Bulga'
BarsConfidence                                                             []
BarsStart                                                       

In [32]:
da = df["BarsStart"][1]

In [ ]:
# Preprocessing
X = []
y = []

print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)